# Setup

In [1]:
%%capture
!pip install --upgrade optuna_integration

In [2]:
import gc
gc.enable()

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import optuna.integration.lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.base import clone
from lightgbm import LGBMClassifier

SEED = 2024

In [3]:
DATA_DIR = '/kaggle/input/bitgrit-crypto-price-prediction'

train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')
sample_sub = pd.read_csv(f'{DATA_DIR}/solution_format.csv')

# Data preparation

In [4]:
TARGET = 'Target'

In [5]:
feats_A = [f'feature_{i}_A' for i in range(1, 10)]
feats_F = [f'feature_{i}_F' for i in range(1, 10)]

CAT_FEATURES = [
    'TR_1_EventInd', 'TR_2_EventInd', 'TR_3_EventInd', 'feature_X_A',
    'feature_10_A', 'feature_10_F_missing', 'feature_10_G'
] + feats_A + feats_F

In [6]:
def preprocess_data(df):
    fill_zero_cols = ['TR_1_EventInd', 'TR_2_EventInd', 'TR_3_EventInd']
    df[fill_zero_cols] = df[fill_zero_cols].fillna(0).astype('int')
    
#     fill_neg1_cols = ['index_1', 'index_2', 'index_3']
#     df[fill_neg1_cols] = df[fill_neg1_cols].fillna(-1).astype('int')
    
    df['feature_10_F_missing'] = df.feature_10_F.isna().astype('int')
    df['feats_A_sum'] = df[feats_A].sum(axis=1)
    df['feats_F_sum'] = df[feats_F].sum(axis=1)
    
    return df


train = preprocess_data(train.copy())
test = preprocess_data(test.copy())

In [7]:
FEATURES = [f for f in test.columns if f not in ('ID', 'feature_10_F')]

In [8]:
# LightGBM dataset
dtrain = lgb.Dataset(
    data=train[FEATURES],
    label=train[TARGET],
    feature_name=FEATURES,
    categorical_feature=CAT_FEATURES)

# LightGBMTuner - tuning

In [9]:
base_params = {
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.01,
    'boosting_type': 'gbdt',
    'force_row_wise': True,
    'verbosity': -1,
    'n_jobs': -1,
    'deterministic': True,
    'random_state': SEED
}

early_stopping = lgb.early_stopping(
    stopping_rounds=100,
    first_metric_only=True,
    verbose=False,
    min_delta=1e-4)

In [10]:
BUDGET = 60 * 60 * 4
NUM_FOLDS = 7

tuner = lgb.LightGBMTunerCV(
    time_budget=BUDGET,
    optuna_seed=SEED,
    params=base_params,
    train_set=dtrain,
    num_boost_round=10000,
    folds=StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=SEED),
    seed=SEED,
    feature_name=FEATURES,
    categorical_feature=CAT_FEATURES,
    callbacks=[early_stopping])

[I 2024-05-29 17:33:39,639] A new study created in memory with name: no-name-0285906c-a0c6-4dca-a7e5-1ba3fe9fec7d


In [11]:
%%time
tuner.run()

min_child_samples, val_score: 0.834031: 100%|##########| 5/5 [10:27<00:00, 125.60s/it]

CPU times: user 2h 3min 50s, sys: 1h 3min 8s, total: 3h 6min 59s
Wall time: 2h 2min 48s


# Cross validation + Thresholding

In [12]:
def probs_to_labels(probs, threshold=0.5):
    return (probs >= threshold).astype('int')

In [13]:
def get_best_threshold(y_true, y_probs):
    candidates = np.arange(0.001, 0.999, 0.001)
    scores = [
        f1_score(y_true, probs_to_labels(y_probs, t)) 
        for t in candidates
    ]
    
    return candidates[np.argmax(scores)]

In [14]:
def custom_cv(estimator, seed=SEED, verbose=True):
    oof_preds = {}
    test_preds = {}
    scores = []
    
    X, y = train[FEATURES], train[TARGET]
    X_test = test[FEATURES]
    
    cv = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=seed)
    for fold, (train_ids, val_ids) in enumerate(cv.split(X, y)):
        X_train, y_train = X.iloc[train_ids], y.iloc[train_ids]
        X_val, y_val = X.iloc[val_ids], y.iloc[val_ids]
        
        model = clone(estimator)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            callbacks=[early_stopping])
        
        val_probs = model.predict_proba(X_val)[:, 1]
        best_threshold = get_best_threshold(y_val, val_probs)
        val_preds = probs_to_labels(val_probs, best_threshold)
        oof_preds.update(dict(zip(val_ids, val_preds)))
        
        test_probs = model.predict_proba(X_test)[:, 1]
        test_preds[f'fold{fold}'] = probs_to_labels(test_probs, best_threshold)
        
        
        f1_best = f1_score(y_val, val_preds)
        scores.append(f1_best)
        if verbose:
            auc = roc_auc_score(y_val, val_probs)
            f1 = f1_score(y_val, probs_to_labels(val_probs, 0.5))
            print(f'Fold #{fold}: AUC = {auc:.5f}, F1 @0.5 = {f1: .5f}, F1 @best = {f1_best:.5f}' \
                  f' ({model.best_iteration_:>4} rounds, best threshold: {best_threshold:.3f})')
            
        _ = gc.collect()
    
    test_preds = pd.DataFrame.from_dict(test_preds)
    test_preds['mode'] = test_preds.mode(axis=1)[0].astype('int')  # mode of fold-wise predictions
    oof_preds = pd.Series(oof_preds).sort_index()
    
    print(f'\nOOF F1 @best: {f1_score(y, oof_preds):.5f}, ' \
          f'Avg F1 @best: {np.mean(scores):.5f} +/- {np.std(scores):.5f}')
    
    return oof_preds, test_preds

In [15]:
%%time
model = LGBMClassifier(**tuner.best_params, n_estimators=10000)
op, tp = custom_cv(model)

Fold #0: AUC = 0.89112, F1 @0.5 =  0.83660, F1 @best = 0.83857 ( 233 rounds, best threshold: 0.459)
Fold #1: AUC = 0.80290, F1 @0.5 =  0.78170, F1 @best = 0.79238 ( 154 rounds, best threshold: 0.386)
Fold #2: AUC = 0.84505, F1 @0.5 =  0.81466, F1 @best = 0.81616 ( 161 rounds, best threshold: 0.490)
Fold #3: AUC = 0.84748, F1 @0.5 =  0.79303, F1 @best = 0.80913 ( 281 rounds, best threshold: 0.465)
Fold #4: AUC = 0.83226, F1 @0.5 =  0.78242, F1 @best = 0.81363 ( 377 rounds, best threshold: 0.360)
Fold #5: AUC = 0.80062, F1 @0.5 =  0.75274, F1 @best = 0.78131 ( 249 rounds, best threshold: 0.343)
Fold #6: AUC = 0.84429, F1 @0.5 =  0.77996, F1 @best = 0.80233 ( 793 rounds, best threshold: 0.293)

OOF F1 @best: 0.80703, Avg F1 @best: 0.80764 +/- 0.01700
CPU times: user 56.9 s, sys: 0 ns, total: 56.9 s
Wall time: 57 s


In [16]:
def create_submission_files(preds, notebook='06'):
    for col in preds.columns:
        sub = sample_sub.copy()
        sub[TARGET] = preds[col]
        sub.to_csv(f'nb{notebook}_{col}.csv', index=False)

In [17]:
create_submission_files(tp)

In [18]:
!head nb06_mode.csv

ID,Target
2661,1
2662,1
2663,1
2664,1
2665,1
2666,1
2667,1
2668,1
2669,1
